<html>
<center>
</h4>
<font color='blod' size="+2">Task Description!</font></h4>
</center>
<pIn this Notebook, we will fine-tune the models BERT and Disease BERT (our model) for Named Entity Recognition (NER) on the NCBI Disease corpora.

<!-- TODO(b/319266067) remove temporary advert after a few weeks. -->
<div class="markdown-google-sans">
  <h2>Biomedical corpora : </h2>
  <ul>
  <li><a href="https://huggingface.co/datasets/ncbi_disease#dataset-card-for-ncbi-disease">NCBI Disease</a></li>
  <p>This dataset contains the disease name and concept annotations of the NCBI disease corpus, a collection of 793 PubMed abstracts fully annotated at the mention and concept level to serve as a research resource for the biomedical natural language processing community.</p>
  <div>
  <center>
  <table border="1">
  <tr>
    <th>Dataset</th>
    <th>Train</th>
    <th>Validation</th>
    <th>Test</th>
  </tr>
  <tr>
    <td>Instances</td>
    <td>5433</td>
    <td>924</td>
    <td>941</td>
  </tr>
</table>
</div>
 
   </div>
  </ul>

</div>
</html>

# BERT

<font color='green'>Preparing the data</font>
<p>First things, we need a dataset suitable for token classification. In this section we will use the NCBI Disease data.</p>
<p>To load the NCBI Disease  dataset, we use the load_dataset() method from the huggingface 🤗 Datasets library</p>

In [1]:
!pip install datasets
!pip install evaluate
!pip install transformers
!pip install seqeval


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=de84cd324298fcadfcfbaa11b5e534fc4173e26185d7a00c59fb0fe749ba9621
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [2]:

import numpy as np
import torch
from accelerate import Accelerator
from transformers import TrainingArguments
from transformers import Trainer
from transformers import AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer
from datasets import load_dataset
import random
import evaluate
random.seed(42)

2024-07-25 18:29:36.423420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 18:29:36.423543: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 18:29:36.557716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
from datasets import load_dataset

raw_datasets = load_dataset("ncbi_disease")

/opt/conda/lib/python3.10/site-packages/datasets/load.py:1491: FutureWarning: The repository for ncbi_disease contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/ncbi_disease
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating train split:   0%|          | 0/5433 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/924 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/941 [00:00<?, ? examples/s]

In [4]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 924
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 941
    })
})

In [5]:
print(raw_datasets["train"][0]["tokens"])

['Identification', 'of', 'APC2', ',', 'a', 'homologue', 'of', 'the', 'adenomatous', 'polyposis', 'coli', 'tumour', 'suppressor', '.']


In [6]:
ner_feature = raw_datasets["train"].features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-Disease', 'I-Disease'], id=None), length=-1, id=None)

In [7]:
label_names = ner_feature.feature.names
label_names

['O', 'B-Disease', 'I-Disease']

In [8]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [9]:
inputs = tokenizer(raw_datasets["train"][0]["tokens"], is_split_into_words=True)
print(inputs.tokens())

['[CLS]', 'identification', 'of', 'ap', '##c', '##2', ',', 'a', 'homo', '##logue', 'of', 'the', 'aden', '##oma', '##tou', '##s', 'poly', '##po', '##sis', 'coli', 'tu', '##mour', 'suppress', '##or', '.', '[SEP]']


In [10]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 924
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 941
    })
})

In [11]:
raw_datasets["train"][0]["tokens"]

['Identification',
 'of',
 'APC2',
 ',',
 'a',
 'homologue',
 'of',
 'the',
 'adenomatous',
 'polyposis',
 'coli',
 'tumour',
 'suppressor',
 '.']

In [12]:
raw_datasets["train"][0]["ner_tags"]

[0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 2, 0, 0]

In [13]:
len(inputs.tokens())

26

In [14]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 924
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 941
    })
})

In [15]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True,is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [16]:
tokenized_datasets = raw_datasets.map(tokenize_and_align_labels,batched=True)

Map:   0%|          | 0/5433 [00:00<?, ? examples/s]

Map:   0%|          | 0/924 [00:00<?, ? examples/s]

Map:   0%|          | 0/941 [00:00<?, ? examples/s]

In [17]:
example = tokenized_datasets['train'][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
len(tokens),len(tokenized_datasets['train'][0]['labels'])

(26, 26)

In [18]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 924
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 941
    })
})

In [19]:
print(tokenized_datasets['train'][0]['labels'])

[-100, 0, 0, 0, -100, -100, 0, 0, 0, -100, 0, 0, 1, -100, -100, -100, 2, -100, -100, 2, 2, -100, 0, -100, 0, -100]


In [20]:
print(tokens)

['[CLS]', 'identification', 'of', 'ap', '##c', '##2', ',', 'a', 'homo', '##logue', 'of', 'the', 'aden', '##oma', '##tou', '##s', 'poly', '##po', '##sis', 'coli', 'tu', '##mour', 'suppress', '##or', '.', '[SEP]']


In [21]:
for i,j in zip(tokenized_datasets['train'][0]['labels'],tokens):
    print(i,j)

-100 [CLS]
0 identification
0 of
0 ap
-100 ##c
-100 ##2
0 ,
0 a
0 homo
-100 ##logue
0 of
0 the
1 aden
-100 ##oma
-100 ##tou
-100 ##s
2 poly
-100 ##po
-100 ##sis
2 coli
2 tu
-100 ##mour
0 suppress
-100 ##or
0 .
-100 [SEP]


In [22]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokenized_input

{'input_ids': [101, 8720, 1997, 9706, 2278, 2475, 1010, 1037, 24004, 24277, 1997, 1996, 16298, 9626, 24826, 2015, 26572, 6873, 6190, 27441, 10722, 20360, 16081, 2953, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [23]:
tokenized_datasets=tokenized_datasets.remove_columns(['id', 'tokens', 'ner_tags'])
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 5433
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 924
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 941
    })
})

In [24]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [25]:
metric = evaluate.load("seqeval")

In [26]:
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [27]:
labels = raw_datasets["train"][0]["ner_tags"]
labels = [label_names[i] for i in labels]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Disease',
 'I-Disease',
 'I-Disease',
 'I-Disease',
 'O',
 'O']

In [28]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [29]:
id2label

{0: 'O', 1: 'B-Disease', 2: 'I-Disease'}

In [30]:
label2id

{'O': 0, 'B-Disease': 1, 'I-Disease': 2}

In [31]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()


#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

Using device: cuda

Tesla P100-PCIE-16GB
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [32]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    "Jaafer/diseaseBert_old_token",num_labels=len(id2label),
    id2label=id2label,
    label2id=label2id,
).to(device)

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at Jaafer/diseaseBert_old_token and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
#just check the number of labels
model.config.num_labels

3

In [34]:
args = TrainingArguments(
    "disease",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=2e-5,
    optim='adamw_torch',
    num_train_epochs=10,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    
)
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.061138,0.752465,0.794792,0.773050,0.978283
2,0.088600,0.052756,0.770335,0.838542,0.802993,0.982120
3,0.019700,0.057555,0.798362,0.812500,0.805369,0.981998
4,0.019700,0.073222,0.810891,0.853125,0.831472,0.982039
5,0.007100,0.082845,0.810496,0.868750,0.838612,0.982610
6,0.003200,0.086610,0.804497,0.857292,0.830055,0.982447
7,0.003200,0.093553,0.807580,0.865625,0.835596,0.982324
8,0.001300,0.099653,0.806641,0.860417,0.832661,0.982814
9,0.000700,0.101342,0.819541,0.856250,0.837494,0.983018
10,0.000700,0.102634,0.812133,0.864583,0.837538,0.982773


TrainOutput(global_step=3400, training_loss=0.017841736200977773, metrics={'train_runtime': 572.6835, 'train_samples_per_second': 94.869, 'train_steps_per_second': 5.937, 'total_flos': 1920408689557692.0, 'train_loss': 0.017841736200977773, 'epoch': 10.0})

In [62]:
disease_ontology_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 11522
    })
})

In [63]:
disease_ontology_dataset["train"]['text'][0]

'angiosarcoma is a vascular cancer that derives_from the cells that line the walls of blood vessels or lymphatic vessels.'

In [60]:
from huggingface_hub import notebook_login

notebook_login()

In [66]:
disease_ontology_dataset.push_to_hub("disease_ontology_dataset")


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/12 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Jaafer/disease_ontology_dataset/commit/c566577d48634d70f6beb53c6c0d83a09c02c246', commit_message='Upload dataset', commit_description='', oid='c566577d48634d70f6beb53c6c0d83a09c02c246', pr_url=None, pr_revision=None, pr_num=None)